In [7]:
import mysql.connector
import sys

try:
    conn = mysql.connector.connect(
        user='root',
        password='',
        host='localhost',
        database='car_price',
        port=3306
    )

except mysql.connector.Error as e:
    print('Error %s' % e)
    
else:
    print('Berhasil Terkoneksi dengan MySQL server')
    conn.close()

Berhasil Terkoneksi dengan MySQL server


In [8]:
import mysql.connector
import sys

try:
    conn = mysql.connector.connect(
        user='root',
        password='',
        host='localhost', 
        database='car_price',
        port=3306
    )

    sql = "SELECT `Car Make`, `Car Model`, `Year`, `Engine Size (L)`, `Horsepower`, `Torque (lb-ft)`, `0-60 MPH Time (seconds)`, `Price (in USD)` FROM sport_car_price"
    
    try:
        cur = conn.cursor()
        cur.execute(sql)
            
        results = cur.fetchall()
        
        print("{:<15}{:<25}{:<10}{:<15}{:<15}{:<15}{:<15}{:<10}".format('Car Make','Car Model','Year','Engine Size','Horsepower','Torque','0-60 MPH','Price'))
        
        for row in results:
            car_make = row[0]
            car_model = row[1] 
            year = row[2]
            engine_size = row[3]
            horsepower = row[4]
            torque = row[5]
            zero_to_60 = row[6] 
            price = row[7]

            print("{:<15}{:<25}{:<10}{:<15}{:<15}{:<15}{:<15}{:<10}".format(car_make,car_model,year,engine_size,horsepower,torque,zero_to_60,price))

                            
    except mysql.connector.DataError:
        print("Pengambilan data error")
        
    else:
        cur.close()
        
except mysql.connector.Error as e:
    print('Gagal koneksi ke MySQL: %s' % e)

else:
    print('Berhasil koneksi ke MySQL')
    conn.close()

Car Make       Car Model                Year      Engine Size    Horsepower     Torque         0-60 MPH       Price     
Porsche        911                      2022      3              379            331            4              101,200   
Lamborghini    Huracan                  2021      5.2            630            443            2.8            274,390   
Ferrari        488 GTB                  2022      3.9            661            561            3              333,750   
Audi           R8                       2022      5.2            562            406            3.2            142,700   
McLaren        720S                     2021      4              710            568            2.7            298,000   
BMW            M8                       2022      4.4            617            553            3.1            130,000   
Mercedes-Benz  AMG GT                   2021      4              523            494            3.8            118,500   
Chevrolet      Corvette         

EKSPLORASI DATA

Memeriksa jumlah baris dan kolom
Menampilkan data teratas
Menampilkan tipe data tiap kolom
Memeriksa missing value / data kosong pada setiap kolom 

In [20]:
import mysql.connector
from sqlalchemy import create_engine
import pandas as pd

try:
    conn = mysql.connector.connect(
        user='root',
        password='', 
        host='localhost',
        database='car_price' 
  )
  
    sql = "SELECT * FROM sport_car_price"

    cur = conn.cursor()
    cur.execute(sql)

    results = cur.fetchall()

    df = pd.DataFrame(results, columns=[col[0] for col in cur.description])

    df['Country'] = df['Car Make'].apply(lambda nama: {
        'Porsche': 'Jerman',
        'Lamborghini': 'Italia',
        'Ferrari': 'Italia',
        'Audi': 'Jerman',
        'McLaren': 'Inggris',
        'BMW': 'Jerman',
        'Mercedes-Benz': 'Jerman',
        'Mercedes-AMG': 'Jerman',
        'Chevrolet': 'Amerika',
        'Ford': 'Amerika',
        'Nissan': 'Jepang',
        'Aston Martin': 'Inggris',
        'Bugatti': 'Prancis',
        'Dodge': 'Amerika',
        'Jaguar': 'Inggris',
        'Koenigsegg': 'Swedia',
        'Lexus': 'Jepang',
        'Lotus': 'Inggris',
        'Maserati': 'Italia',
        'Pagani': 'Italia',
        'Rimac': 'Kroasia',
        'Rolls-Royce': 'Inggris',
        'Tesla': 'Amerika',
        'Toyota': 'Jepang',
        'TVR': 'Inggris',
        'Ultima': 'Inggris',
        'W Motors': 'Lebanon',
        'Alpine': 'Prancis',
        'Alfa Romeo': 'Italia',
        'Bentley': 'Inggris',
        'Subaru': 'Jepang',
        'Acura': 'Jepang',
        'Ariel': 'Inggris',
        'Kia': 'Korea Selatan',
        'Pininfarina': 'Italia',
        'Shelby': 'Amerika',
        'Mazda': 'Jepang',
    }.get(nama, 'Tidak Diketahui'))

    engine = create_engine('mysql+mysqlconnector://root:@localhost/car_price')
    df.to_sql(name='sport_car_price_updated', con=engine, if_exists='replace', index=False)
    
    baris, kolom = df.shape
    print(f"Data frame memiliki {baris} baris dan {kolom} kolom\n")
    
    jumlah_data_teratas = df.head().to_string(index=False) 
    print(f"Jumlah data teratas:\n{jumlah_data_teratas}\n")
    
    tipe_data = df.dtypes
    print(f"Tipe data setiap kolom:\n{tipe_data}\n")
    
    data_kosong = df.isnull().sum().sum()
    print(f"Jumlah Data kosong pada setiap kolom:{data_kosong}\n")
    
    
except mysql.connector.Error as e:
  print("Error %s" % e)
  
finally:
    if 'conn' in locals() and conn.is_connected():
        conn.close()
        print('Berhasil terkoneksi dengan MySQL server')

Data frame memiliki 1007 baris dan 9 kolom

Jumlah data teratas:
   Car Make Car Model  Year Engine Size (L) Horsepower Torque (lb-ft) 0-60 MPH Time (seconds) Price (in USD) Country
    Porsche       911  2022               3        379            331                       4        101,200  Jerman
Lamborghini   Huracan  2021             5.2        630            443                     2.8        274,390  Italia
    Ferrari   488 GTB  2022             3.9        661            561                       3        333,750  Italia
       Audi        R8  2022             5.2        562            406                     3.2        142,700  Jerman
    McLaren      720S  2021               4        710            568                     2.7        298,000 Inggris

Tipe data setiap kolom:
Car Make                   object
Car Model                  object
Year                        int64
Engine Size (L)            object
Horsepower                 object
Torque (lb-ft)             object
0-60

DATA CLEANING

Cek Outlier dan Visualisasi Distribusi pada setiap kolom

In [ ]:
import mysql.connector
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

try:
    conn = mysql.connector.connect(
    user='root',
    password='', 
    host='localhost',
    database='car_price' 
  )
  
    sql = "SELECT * FROM sport_car_price"

    cur = conn.cursor()
    cur.execute(sql)

    results = cur.fetchall()

    df = pd.DataFrame(results, columns=[col[0] for col in cur.description])
    
    z = np.abs(stats.zscore(df._get_numeric_data()))
    df = df[(z < 3).all(axis=1)]
    print(df.to_string(index=False))
   
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(12, 12))

    for i, col in enumerate(df.columns):
        sns.histplot(df[col], ax=axes[i//3, i%3])
        axes[i//3, i%3].set_title(f'Distribusi dari {col}')
    
    plt.tight_layout()
    plt.show()
      
    
except mysql.connector.Error as e:
  print("Error %s" % e)
  
finally:
  conn.close()
  print('Berhasil terkoneksi dengan MySQL server')

ANALISIS DATA

Latar Belakang dari Dataset Penjualan Mobil :
- Analisis Pasar Otomotif: Memungkinkan pemahaman mendalam tentang perilaku konsumen dan tren pasar otomotif. Analisis ini dapat melibatkan faktor-
  faktor seperti permintaan pasar, preferensi konsumen, dan respon terhadap perubahan ekonomi.
- Keterkaitan Fitur Mobil: Menyediakan informasi tentang hubungan antara berbagai fitur mobil, seperti model, tahun keluaran, daya kuda (hp), berat, dan
  harga. Hal ini dapat membantu mengidentifikasi pola atau korelasi yang dapat digunakan untuk pengambilan keputusan lebih lanjut.
- Evaluasi Performa Merek/Model: Memungkinkan pembandingan performa berbagai merek atau model mobil. Analisis ini dapat melibatkan penilaian terhadap
  popularitas, pertumbuhan penjualan, atau dominasi pasar oleh beberapa merek atau model tertentu.
- Pemahaman Variabilitas Harga: Memberikan wawasan tentang variasi harga mobil berdasarkan atribut-atribut tertentu. Ini dapat melibatkan penelusuran
  perbedaan harga antara mobil dengan spesifikasi yang serupa atau pengidentifikasian faktor-faktor utama yang mempengaruhi harga.

Tujuan dari Dataset Penjualan Mobil :
- Melakukan analisis tren harga mobil berdasarkan model dan tahun keluaran. Dengan melihat perubahan harga rata-rata tiap model mobil sepanjang
  beberapa tahun, Anda bisa mempelajari bagaimana harga mobil cenderung berubah seiring waktu.
- Membuat model prediksi harga mobil berdasarkan spesifikasinya seperti model, tahun keluaran, hp, dan berat. Anda bisa membuat model regresi untuk
  memprediksi harga mobil baru berdasarkan spesifikasinya.
- Melakukan segmentasi pasar dan analisis preferensi konsumen terhadap model dan spesifikasi mobil tertentu berdasarkan harganya. Anda bisa melihat
  kelas mobil dan spesifikasi yang paling laku di tiap segmen harga.
- Menganalisis korelasi antara spesifikasi mobil seperti hp dan berat terhadap harganya untuk memahami faktor apa saja yang berpengaruh terhadap
  penetapan harga.
- Memvisualisasikan data untuk melihat distribusi dan perbandingan harga mobil berdasarkan model, tahun, dan spesifikasi lainnya.